In [36]:
from dotenv import load_dotenv
import os
import pandas as pd
from openai import OpenAI
from pydantic import BaseModel, Field
from src.dataimport import list_files_with_extension_directory, load_text
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

# load files

In [15]:
TXT_FILES_PATH = 'data/original/brat-project-final/'
JSON_FILES_PATH = 'data/transformed/'

In [16]:
txt_files_directory_list = list_files_with_extension_directory(TXT_FILES_PATH, '.txt')
# txt_files_directory_list

json_files_directory_list = list_files_with_extension_directory(JSON_FILES_PATH, '.json')
# json_files_directory_list

print(f"Anzahl Text-Dateien: {len(txt_files_directory_list)}")
print(f"Anzahl Brat-Dateien: {len(json_files_directory_list)}")

Anzahl Text-Dateien: 402
Anzahl Brat-Dateien: 402


In [17]:
# create dataframe with file names
df = pd.DataFrame()
df['txt_path'] = txt_files_directory_list
df['json_path'] = json_files_directory_list
df['txt_file'] = df['txt_path'].apply(lambda x: os.path.basename(x))
df['json_file'] = df['json_path'].apply(lambda x: os.path.basename(x))
df['txt'] = df['txt_path'].apply(load_text)
df['json'] = df['json_path'].apply(load_text)

print(df.shape)
df.head()

# save to csv
#df.to_csv('dataframe.csv', index=False)
# load dataframe
# df = pd.read_csv('dataframe.csv')
# df.head()

(402, 6)


,txt_path,json_path,txt_file,json_file,txt,json
0,data/original/brat-project-final/essay001.txt,data/transformed/essay001.json,essay001.txt,essay001.json,Should students be taught to compete or to coo...,"{\n ""MajorClaims"": {\n ""MC1"": ""we should a..."
1,data/original/brat-project-final/essay002.txt,data/transformed/essay002.json,essay002.txt,essay002.json,More people are migrating to other countries t...,"{\n ""MajorClaims"": {\n ""MC1"": ""they are ab..."
2,data/original/brat-project-final/essay003.txt,data/transformed/essay003.json,essay003.txt,essay003.json,International tourism is now more common than ...,"{\n ""MajorClaims"": {\n ""MC1"": ""it has cont..."
3,data/original/brat-project-final/essay004.txt,data/transformed/essay004.json,essay004.txt,essay004.json,International tourism is now more common than ...,"{\n ""MajorClaims"": {\n ""MC1"": ""this indust..."
4,data/original/brat-project-final/essay005.txt,data/transformed/essay005.json,essay005.txt,essay005.json,Living and studying overseas\n\nIt is every st...,"{\n ""MajorClaims"": {\n ""MC1"": ""one who stu..."


# train test split

In [18]:
# Split the dataframe into training and test sets
train_df, test_df = train_test_split(df, train_size=40, random_state=42)

# Display the first few rows of the training and test sets
print(f"Training DataFrame: {train_df.shape}")
print(f"\nTest DataFrame: {test_df.shape}")

Training DataFrame: (40, 6)

Test DataFrame: (362, 6)


# load prompt templates

In [19]:
PROMPTS_PATH = 'prompts/final-prompts/'

In [20]:
prompt_files_directory_list = list_files_with_extension_directory(PROMPTS_PATH, '.txt')
prompt_files_directory_list
prompt_files_list = [os.path.basename(x) for x in prompt_files_directory_list]
# remove the .txt extension
prompt_names = [x.split('.')[0] for x in prompt_files_list]

prompt_df = pd.DataFrame()
# get the file name without the extension from prompt_files, 'str' object has no attribute 'path'
prompt_df['prompt_name'] = prompt_names
prompt_df['prompt_txt'] = prompt_files_directory_list
prompt_df['prompt_txt'] = prompt_df['prompt_txt'].apply(load_text)
print(F"Es gibt {prompt_df.shape[0]} Prompts")
# prompt_df

Es gibt 20 Prompts


In [21]:
# get the API key from the .env file
load_dotenv()
llama_api = os.getenv("HUGGINGFACE_TOKEN")

model_id = "meta-llama/Llama-3.2-3B-Instruct"
# model_id = "meta-llama/Llama-3.3-70B-Instruct" # requires HugginFace Pro subscription

# Function to calculate token count
def calculate_token_count(prompt):
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenized_prompt = tokenizer(prompt, return_tensors='pt') # pt for PyTorch tensors
    return tokenized_prompt.input_ids.size(1)

# Apply the function to the 'prompt' column and create a new column 'token_count'
prompt_df['token_count'] = prompt_df['prompt_txt'].apply(calculate_token_count)

prompt_df.sort_values(by='token_count')

,prompt_name,prompt_txt,token_count
19,zero-shot,You will be given a text. Extract the argument...,102
18,zero-shot-persona,You are a expert in Argument Mining and theref...,126
16,zero-shot-cot,You will be given a text. Extract the argument...,470
17,zero-shot-persona-cot,You are a expert in Argument Mining and theref...,494
15,one-shot,You will be given a text. Extract the argument...,1550
14,one-shot-persona,You are a expert in Argument Mining and theref...,1562
12,one-shot-cot,You will be given a text. Extract the argument...,1907
13,one-shot-persona-cot,You are a expert in Argument Mining and theref...,1931
3,few-shot-10,You will be given a text. Extract the argument...,12088
2,few-shot-10-persona,You are a expert in Argument Mining and theref...,12112


In [22]:
# structured output 
class ArgumentativeRelation(BaseModel):
    """Argumentative relation between the origin and target"""
    origin_id: str = Field(description="ID of the claim or premise")
    relation_type: str = Field(description="Type of relation (e.g., 'For', 'Against', 'Support', 'Attack')")
    target_id: str = Field(description="ID of the target (e.g., Major Claim ID)")

class ArgumentMiningExtraction(BaseModel):
    """Extraction of argument components and relations from a text"""
    major_claims: dict[str, str] = Field(description="Dictionary of major claims with their IDs as keys and text as values")
    claims: dict[str, str] = Field(description="Dictionary of claims with their IDs as keys and text as values")
    premises: dict[str, str] = Field(description="Dictionary of premises with their IDs as keys and text as values")
    argumentative_relations: list[ArgumentativeRelation] = Field(description="List of argumentative relations between origin and target")

    
# Quellen Structured Outputs:
# - https://platform.openai.com/docs/guides/structured-outputs
# - https://python.langchain.com/docs/concepts/structured_outputs/
# - https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html -->Structured output

# allgemeiner Test

In [24]:
load_dotenv()
openai_api = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api)

llm = ChatOpenAI(
    model="gpt-4o-mini",
    #max_tokens=1024,
    #max_tokens_input=1024,
    temperature=0,
    timeout=None,
    # max_retries=2,
    api_key=openai_api,
    #response_format=ArgumentMiningExtraction
    )

In [25]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="J'aime la programmation.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 31, 'total_tokens': 37, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'stop', 'logprobs': None}, id='run-10370652-6415-4656-9498-3521cebcd09c-0', usage_metadata={'input_tokens': 31, 'output_tokens': 6, 'total_tokens': 37, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [26]:
print(ai_msg.content)

J'aime la programmation.


In [27]:
# Token usage tracking
ai_msg.usage_metadata

# Quelle: 
# - https://python.langchain.com/docs/how_to/chat_token_usage_tracking/
# - https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html

{'input_tokens': 31,
 'output_tokens': 6,
 'total_tokens': 37,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

# Argument mining Test

In [34]:
# Beispiel aus Testdatensatz
test_data_example = test_df.sample(1, random_state=42)
test_data_example = test_data_example['txt'].values[0]
print(test_data_example) 

The precondition of doing research by professors

Nowadays, many professors conduct research while teaching in colleges or universities. Although research could bring funding and latest achievements in the field, the research takes up too much teaching time. As far as I am concerned, professors should spend more time on preparing courses than research.
To begin with, it is vital that professors should assist students to acquire knowledge. The professors’ duty is to nurture students, and prepare excellent courses. If professors spend a lot of time on research, he will cut the time on preparing courses is reduced, which may decrease the quality of classes. The reason why people enter a university is that they want to learn cutting-edged technologies. In this way, professors should provide well-preparation courses to convey the state-of-art knowledge for college and university students. Therefore, professors should pay more attention on how to teach students rather than conduct research.


In [32]:
# zero-shot als test-prompt
zero_shot_prompt = prompt_df.loc[prompt_df['prompt_name'] == 'zero-shot', 'prompt_txt'].values[0]
zero_shot_prompt

'You will be given a text. Extract the argumentative units “major claim”, “claim”, and “premise” as parts from the text. Also extract the argumentative relationships between the units. Claims can be “for” or “against” the major claims. Premises, on the other hand, can “support” or “attack” a claim or another premise. It is possible that there are several major claims. Return only the argumentative units and relationships between them as a JSON object.'

In [40]:
load_dotenv()
openai_api = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api)

In [48]:
llm_struc_output = ChatOpenAI(
    model="gpt-4o-mini",
    #max_tokens=1024,
    #max_tokens_input=1024,
    temperature=0,
    timeout=None,
    # max_retries=2,
    api_key=openai_api,
    #response_format=ArgumentMiningExtraction,
    model_kwargs={"structured_output": True,
                  "structured_output_type": ArgumentMiningExtraction}
    )

# llm_struc_output

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002104F1916A0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002104F1934A0>, root_client=<openai.OpenAI object at 0x000002104F15B050>, root_async_client=<openai.AsyncOpenAI object at 0x000002104F1915E0>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={'structured_output': True, 'structured_output_type': <class '__main__.ArgumentMiningExtraction'>}, openai_api_key=SecretStr('**********'))

In [42]:
# template
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "{system_message}"),
        ("user", "Text: {argument_text}"),
    ]
)

# print example for one prompt template
print(
    prompt_template.invoke(
        {
            "system_message": zero_shot_prompt,
            "argument_text": test_data_example,
        }
    )
)

messages=[SystemMessage(content='You will be given a text. Extract the argumentative units “major claim”, “claim”, and “premise” as parts from the text. Also extract the argumentative relationships between the units. Claims can be “for” or “against” the major claims. Premises, on the other hand, can “support” or “attack” a claim or another premise. It is possible that there are several major claims. Return only the argumentative units and relationships between them as a JSON object.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Text: The precondition of doing research by professors\n\nNowadays, many professors conduct research while teaching in colleges or universities. Although research could bring funding and latest achievements in the field, the research takes up too much teaching time. As far as I am concerned, professors should spend more time on preparing courses than research.\nTo begin with, it is vital that professors should assist students to acquire kn

In [39]:
llm_chain = prompt_template | llm
llm_chain

ChatPromptTemplate(input_variables=['argument_text', 'system_message'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['system_message'], input_types={}, partial_variables={}, template='{system_message}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['argument_text'], input_types={}, partial_variables={}, template='Text: {argument_text}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002104EFF90A0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002104EFFADB0>, root_client=<openai.OpenAI object at 0x000002104C85C980>, root_async_client=<openai.AsyncOpenAI object at 0x000002104EFF9100>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

## zero-shot

In [43]:
zero_shot_answ = llm_chain.invoke(
    {
        "system_message": zero_shot_prompt,
        "argument_text": test_data_example,
    }
)
zero_shot_answ

AIMessage(content='```json\n{\n  "units": {\n    "major_claims": [\n      "Professors should spend more time on preparing courses than research.",\n      "Conducting research is helpful for professors and students to some extent."\n    ],\n    "claims": [\n      {\n        "claim": "Professors should assist students to acquire knowledge.",\n        "relationship": "support",\n        "to": "Professors should spend more time on preparing courses than research."\n      },\n      {\n        "claim": "If professors spend a lot of time on research, the quality of classes may decrease.",\n        "relationship": "support",\n        "to": "Professors should spend more time on preparing courses than research."\n      },\n      {\n        "claim": "If professors are busy in doing research, it would ruin the reputation of universities.",\n        "relationship": "support",\n        "to": "Professors should spend more time on preparing courses than research."\n      },\n      {\n        "claim": 

In [45]:
print(zero_shot_answ.content)

```json
{
  "units": {
    "major_claims": [
      "Professors should spend more time on preparing courses than research.",
      "Conducting research is helpful for professors and students to some extent."
    ],
    "claims": [
      {
        "claim": "Professors should assist students to acquire knowledge.",
        "relationship": "support",
        "to": "Professors should spend more time on preparing courses than research."
      },
      {
        "claim": "If professors spend a lot of time on research, the quality of classes may decrease.",
        "relationship": "support",
        "to": "Professors should spend more time on preparing courses than research."
      },
      {
        "claim": "If professors are busy in doing research, it would ruin the reputation of universities.",
        "relationship": "support",
        "to": "Professors should spend more time on preparing courses than research."
      },
      {
        "claim": "Spending more time to nurture excellent st

In [46]:
zero_shot_answ.usage_metadata

{'input_tokens': 573,
 'output_tokens': 630,
 'total_tokens': 1203,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

## one shot

In [51]:
# zero-shot als test-prompt
one_shot_prompt = prompt_df.loc[prompt_df['prompt_name'] == 'one-shot', 'prompt_txt'].values[0]
print(one_shot_prompt)

You will be given a text. Extract the argumentative units “major claim”, “claim”, and “premise” as parts from the text. Also extract the argumentative relationships between the units. Claims can be “for” or “against” the major claims. Premises, on the other hand, can “support” or “attack” a claim or another premise. It is possible that there are several major claims. Return only the argumentative units and relationships between them as a JSON object.Here is one example of a text and its corresponding json data:
## Input:
Do you think it is good for teenagers to work while schooling?

In my opinion, it is not the good idea for teenagers to have job while they are still students. Although, many argue that it provide good working experience, but I think it can interfere with their life in various ways. Having jobs would affect the health of the student. It divert their mind from studies and would take away their childhood phase from their life.
A student has to do lots of studies in today

In [70]:
# one-shot
os_template = prompt_template.invoke(
        {
            "system_message": one_shot_prompt,
            "argument_text": test_data_example,
        }
    )
print(os_template)

messages=[SystemMessage(content='You will be given a text. Extract the argumentative units “major claim”, “claim”, and “premise” as parts from the text. Also extract the argumentative relationships between the units. Claims can be “for” or “against” the major claims. Premises, on the other hand, can “support” or “attack” a claim or another premise. It is possible that there are several major claims. Return only the argumentative units and relationships between them as a JSON object.Here is one example of a text and its corresponding json data:\n## Input:\nDo you think it is good for teenagers to work while schooling?\n\nIn my opinion, it is not the good idea for teenagers to have job while they are still students. Although, many argue that it provide good working experience, but I think it can interfere with their life in various ways. Having jobs would affect the health of the student. It divert their mind from studies and would take away their childhood phase from their life.\nA stud

In [71]:
system_msg = os_template.messages[0].content 
# print(system_msg)
user_msg = os_template.messages[1].content
# print(user_msg)

In [72]:
os_answ = llm_chain.invoke(
    {
        "system_message": one_shot_prompt,
        "argument_text": test_data_example,
    }
)

'{\n  "MajorClaims": {\n    "MC1": "professors should spend more time on preparing courses than research",\n    "MC2": "spending more time to nurture excellent students is more consequential to professors, rather than research",\n    "MC3": "the precondition of doing research is growing students’ solid theory fundamental"\n  },\n  "Claims": {\n    "C1": "research could bring funding and latest achievements in the field",\n    "C2": "the research takes up too much teaching time",\n    "C3": "professors should assist students to acquire knowledge",\n    "C4": "the professors’ duty is to nurture students, and prepare excellent courses",\n    "C5": "if professors spend a lot of time on research, the time on preparing courses is reduced",\n    "C6": "this may decrease the quality of classes",\n    "C7": "students want to learn cutting-edged technologies",\n    "C8": "professors should provide well-preparation courses to convey the state-of-art knowledge",\n    "C9": "if professors are busy 

In [74]:
print(os_answ.content)

{
  "MajorClaims": {
    "MC1": "professors should spend more time on preparing courses than research",
    "MC2": "spending more time to nurture excellent students is more consequential to professors, rather than research",
    "MC3": "the precondition of doing research is growing students’ solid theory fundamental"
  },
  "Claims": {
    "C1": "research could bring funding and latest achievements in the field",
    "C2": "the research takes up too much teaching time",
    "C3": "professors should assist students to acquire knowledge",
    "C4": "the professors’ duty is to nurture students, and prepare excellent courses",
    "C5": "if professors spend a lot of time on research, the time on preparing courses is reduced",
    "C6": "this may decrease the quality of classes",
    "C7": "students want to learn cutting-edged technologies",
    "C8": "professors should provide well-preparation courses to convey the state-of-art knowledge",
    "C9": "if professors are busy in doing researc

In [73]:
os_answ.usage_metadata

{'input_tokens': 2018,
 'output_tokens': 1138,
 'total_tokens': 3156,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

# Evaluation

In [82]:
import json
os_data = os_answ.content
os_data = json.loads(os_data)
os_data 

{'MajorClaims': {'MC1': 'professors should spend more time on preparing courses than research',
  'MC2': 'spending more time to nurture excellent students is more consequential to professors, rather than research',
  'MC3': 'the precondition of doing research is growing students’ solid theory fundamental'},
 'Claims': {'C1': 'research could bring funding and latest achievements in the field',
  'C2': 'the research takes up too much teaching time',
  'C3': 'professors should assist students to acquire knowledge',
  'C4': 'the professors’ duty is to nurture students, and prepare excellent courses',
  'C5': 'if professors spend a lot of time on research, the time on preparing courses is reduced',
  'C6': 'this may decrease the quality of classes',
  'C7': 'students want to learn cutting-edged technologies',
  'C8': 'professors should provide well-preparation courses to convey the state-of-art knowledge',
  'C9': 'if professors are busy in doing research, it would ruin the reputation of un

In [92]:
major_claims_df = pd.DataFrame(os_data['MajorClaims'].items(), columns=['ID', 'Text'])
claims_df = pd.DataFrame(os_data['Claims'].items(), columns=['ID', 'Text'])
premises_df = pd.DataFrame(os_data['Premises'].items(), columns=['ID', 'Text'])
relations_df = pd.DataFrame(os_data['ArgumentativeRelations'])

In [89]:
major_claims_df

,ID,Text
0,MC1,professors should spend more time on preparing...
1,MC2,spending more time to nurture excellent studen...
2,MC3,the precondition of doing research is growing ...


In [90]:
claims_df

,ID,Text
0,C1,research could bring funding and latest achiev...
1,C2,the research takes up too much teaching time
2,C3,professors should assist students to acquire k...
3,C4,"the professors’ duty is to nurture students, a..."
4,C5,"if professors spend a lot of time on research,..."
5,C6,this may decrease the quality of classes
6,C7,students want to learn cutting-edged technologies
7,C8,professors should provide well-preparation cou...
8,C9,"if professors are busy in doing research, it w..."
9,C10,students are unsatisfied with the unprepared c...


In [93]:
premises_df

,ID,Text
0,P1,the reason why people enter a university is th...
1,P2,students would not choose these kinds of unive...
2,P3,it is adverse to the whole academic field
3,P4,through to use the equipment and invent their ...
4,P5,electronic students always invent new machines...
5,P6,they can manage the electric theory in practic...
6,P7,some students will be rewarded by investing ne...


In [94]:
relations_df

,Claim,Relation,Target
0,C1,For,MC1
1,C2,For,MC1
2,C3,For,MC1
3,C4,For,MC1
4,C5,supports,C4
5,C6,supports,C5
6,C7,supports,C4
7,C8,For,MC1
8,C9,For,MC2
9,C10,supports,C9
